# A simple chatbot agent

## Prepare the environment

In [1]:
import sys
sys.path.append('../../..')

from autogen import ConversableAgent, register_function

# from inswitch.llm.model import get_openai_model_config
from inswitch.agent.basic import get_chat_agent, get_fixed_reply_agent
from inswitch.util.message import second_last_msg

from usecases.fill.filluc.mockupnerv.session import make_request

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


## The simplied doc providing agent: no RAG in it - only manually selected text

In [2]:
with open("../data/nerve_api_dna.txt", "r") as api_doc:
    content = api_doc.read()

api_doc_provider = get_fixed_reply_agent(
    'api_doc_provider', 
    reply=content
)

moderator = get_fixed_reply_agent(
    name="moderator",
    reply = ""
)


## Here are the API invocation agents

In [3]:
nerv_tool_driver_system_message = '''You are a helpful assistent.
You have access to a tool to call the Nerve DNA API to fulfil the user's intent.
You will get from the context a document of the Nerve DNA API.
Using the document, you will figure out how to call the API, i.e., using what endpoint and method.
You will also need to generate the configuration file as the data of the api call, to fulfil the task assigned to you.
'''

nerv_tool_driver = get_chat_agent(
    "nerv_tool_driver",
    system_message = nerv_tool_driver_system_message
)

nerv_tool_executor = get_fixed_reply_agent(
    "nerv_tool_executor",
    reply = ""
)

register_function(
    make_request,
    caller = nerv_tool_driver,
    executor = nerv_tool_executor,
    description = "This is the functionto send a request to the Nerve API for deploying workloads etc. "
)

nerv_tool_executor.register_nested_chats(
    [
        {
            "recipient": nerv_tool_driver,
            "max_turns": 2,
            "summary_method": second_last_msg
        }
    ],
    trigger = lambda sender: sender not in [nerv_tool_driver]
)



The following parameters of the function 'make_request' with default values are not annotated: 'files', 'workaround'.


## Here we go!
A simple sequence: user_intent -> api_doc_provider -> nerv_tool_executor

Current intent is hard coded.

In [4]:
user_intent = '''My intent is to deploy three docker workloads named "ngix" (version 1.27), 
"neo4j" (version 4.4) and "machine_monitor" (version "1.0")
into the machine with serial number M00002.
'''

intent_provider = get_fixed_reply_agent(
    'intent_provier',
    reply = user_intent
)

chat_result = moderator.initiate_chats(
    [
        {
            "recipient": intent_provider,
            "message": "what do you want?",
            "max_turns": 1,
            "summary_method": "last_msg"
        },
        {
            "recipient": api_doc_provider,
            "message": "Please provide me the api doc",
            "max_turns": 1,
            "summary_method": "last_msg"
        },
        {
            "recipient": nerv_tool_executor,
            "message": "you can see the intent and the knowledge in the context",
            "max_turns": 1,
            "summary_method": "last_msg"
        }
    ]
)



********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to intent_provier):

what do you want?

--------------------------------------------------------------------------------
intent_provier (to moderator):

My intent is to deploy three docker workloads named "ngix" (version 1.27), 
"neo4j" (version 4.4) and "machine_monitor" (version "1.0")
into the machine with serial number M00002.


--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to api_doc_provider):

Please provide me the api doc
Context: 
My intent is to deploy three docker workloads named "ngix" (version 1.27), 
"neo4j" (version 4.4) and "machine_monitor" (ve

## Next steps:

1. Replace current api_doc_provider (a fixed_reply_agent) by a RAG assistant, that extracts from a complete API document the parts that are relevant to DNA and deployment.
2. Add in the beginning of the sequence another agent (or agents) to extract the list of workloads. Eventually, the input intent should be like "I want to be able to monitor the thermal stability of machine M0001"